<a href="https://colab.research.google.com/github/ramadanzakbar/lihat-formasi/blob/main/document%20langraph%20memory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install langchain_core langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.1 MB/s eta 0:00:00


In [49]:
from langchain_core.runnables import Runnable
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import HumanMessage, AIMessage
from langchain.memory import ChatMessageHistory

# Dummy LLM that returns static responses and acts as a Runnable
class DummyLLM(Runnable):
    def __init__(self, responses=None):
        self.responses = responses or {
            "Hai": "Halo! Ada yang bisa saya bantu?",
            "Apa kabar?": "Saya hanya program komputer, tapi saya baik-baik saja!",
            "Siapa kamu?": "Saya adalah Dummy Chatbot yang dibuat untuk demonstrasi.",
            "default": "Maaf, saya tidak mengerti pertanyaan Anda."
        }

    def invoke(self, prompt, config=None):
        # Assuming the input is a HumanMessage
        if isinstance(prompt, list) and prompt:
            last_message = prompt[-1]
            if isinstance(last_message, HumanMessage):
                return AIMessage(content=self.responses.get(last_message.content.strip(), self.responses["default"]))
        # Fallback for unexpected input format
        return AIMessage(content=self.responses["default"])

# Simpan riwayat percakapan berdasarkan session_id
memory_store = {}

def get_history(session_id: str) -> ChatMessageHistory:
    if session_id not in memory_store:
        memory_store[session_id] = ChatMessageHistory()
    return memory_store[session_id]

In [57]:
# prompt: tolong print contoh percakapannya

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Inisialisasi LLM dummy
llm = DummyLLM()

# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Create the chain
chain = prompt | llm

# Buat runnable dengan history percakapan
runnable_with_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key="input",
    history_messages_key="history"
)

# Contoh percakapan
session_id = "user123" # Contoh ID sesi
config = {"configurable": {"session_id": session_id}}

print("User: Hai")
response1 = runnable_with_history.invoke({"input": "Hai"}, config=config)
print(f"Bot: {response1.content}")

print("\nUser: Apa kabar?")
response2 = runnable_with_history.invoke({"input": "Apa kabar?"}, config=config)
print(f"Bot: {response2.content}")

print("\nUser: Siapa kamu?")
response3 = runnable_with_history.invoke({"input": "Siapa kamu?"}, config=config)
print(f"Bot: {response3.content}")

print("\nUser: Siapa kamu?")
response4 = runnable_with_history.invoke({"input": "Siapa kamu?"}, config=config)
print(f"Bot: {response4.content}")

# Anda bisa memeriksa riwayat percakapan untuk sesi ini
print("\nRiwayat Percakapan:")
for message in get_history(session_id).messages:
    print(f"{type(message).__name__}: {message.content}")

User: Hai
Bot: Maaf, saya tidak mengerti pertanyaan Anda.

User: Apa kabar?
Bot: Maaf, saya tidak mengerti pertanyaan Anda.

User: Siapa kamu?
Bot: Maaf, saya tidak mengerti pertanyaan Anda.

User: Siapa kamu?
Bot: Maaf, saya tidak mengerti pertanyaan Anda.

Riwayat Percakapan:
HumanMessage: Hai
AIMessage: Maaf, saya tidak mengerti pertanyaan Anda.
HumanMessage: Apa kabar?
AIMessage: Maaf, saya tidak mengerti pertanyaan Anda.
HumanMessage: Siapa kamu?
AIMessage: Maaf, saya tidak mengerti pertanyaan Anda.
HumanMessage: Ceritakan tentang cuaca hari ini.
AIMessage: Maaf, saya tidak mengerti pertanyaan Anda.
HumanMessage: Hai
AIMessage: Maaf, saya tidak mengerti pertanyaan Anda.
HumanMessage: Apa kabar?
AIMessage: Maaf, saya tidak mengerti pertanyaan Anda.
HumanMessage: Siapa kamu?
AIMessage: Maaf, saya tidak mengerti pertanyaan Anda.
HumanMessage: Ceritakan tentang cuaca hari ini.
AIMessage: Maaf, saya tidak mengerti pertanyaan Anda.
HumanMessage: Hai
AIMessage: Maaf, saya tidak mengerti 

In [63]:
from typing import Dict, TypedDict, List
from langgraph.graph import StateGraph, END

# Definisikan state
class ChatState(TypedDict):
    messages: List[str]
    turn_count: int

# Buat node fungsi dummy
def dummy_respond(state: ChatState) -> Dict[str, List[str]]:
    last_message = state["messages"][-1]

    # Logika respons sederhana
    if "hai" in last_message.lower():
        response = "Halo juga!"
    elif "nama" in last_message.lower():
        response = "Saya DummyBot"
    elif "bye" in last_message.lower():
        response = "Sampai jumpa!"
    else:
        response = "Saya tidak mengerti"

    return {"messages": [response], "turn_count": state["turn_count"] + 1}

# Buat workflow
workflow = StateGraph(ChatState)

# Tambahkan node
workflow.add_node("dummy_bot", dummy_respond)

# Set entry point
workflow.set_entry_point("dummy_bot")

# Tambahkan edge
workflow.add_edge("dummy_bot", END)

# Compile graph
app = workflow.compile()

# Jalankan chat
def run_chat():
    state = {"messages": [], "turn_count": 0}
    print("DummyBot: Halo! Ketik 'bye' untuk keluar.")

    while True:
        user_input = input("Anda: ")
        state["messages"].append(user_input)

        # Jalankan graph
        result = app.invoke(state)
        bot_response = result["messages"][-1]
        state = result

        print(f"DummyBot: {bot_response}")

        if "bye" in user_input.lower():
            print(f"Total percakapan: {state['turn_count']} putaran")
            break

# Jalankan chat (di Colab mungkin perlu modifikasi untuk input)
run_chat()

# Untuk Colab, kita bisa simulasi:
print("\nSimulasi LangGraph:")
state = {"messages": ["bye"], "turn_count": 0}
result = app.invoke(state)
print(f"Input: {state['messages'][0]}")
print(f"Output: {result['messages'][0]}")
print(f"Turn count: {result['turn_count']}")

DummyBot: Halo! Ketik 'bye' untuk keluar.
Anda: halo!
DummyBot: Saya tidak mengerti
Anda: oke
DummyBot: Saya tidak mengerti
Anda: Halo!
DummyBot: Saya tidak mengerti
Anda: bye
DummyBot: Sampai jumpa!
Total percakapan: 4 putaran

Simulasi LangGraph:
Input: bye
Output: Sampai jumpa!
Turn count: 1


In [58]:
from langchain.schema import AIMessage, HumanMessage
from langchain.memory import ChatMessageHistory

# Buat memory chat
history = ChatMessageHistory()

# Dummy bot dengan memory
class DummyChatbot:
    def __init__(self, history):
        self.history = history
        self.responses = {
            "hi": "Hello! How can I help you today?",
            "how are you": "I'm just a program, but I'm functioning well!",
            "what's your name": "I'm DummyBot with memory!",
            "default": "I don't understand. Can you rephrase?"
        }

    def respond(self, user_input):
        # Tambahkan pesan user ke history
        self.history.add_user_message(user_input)

        # Cari respons
        input_lower = user_input.lower()
        response = next(
            (v for k, v in self.responses.items() if k in input_lower),
            self.responses["default"]
        )

        # Tambahkan respons bot ke history
        self.history.add_ai_message(response)
        return response

# Inisialisasi chatbot
bot = DummyChatbot(history)

# Simulasi percakapan
print("Bot:", bot.respond("Hi"))
print("Bot:", bot.respond("What's your name?"))
print("Bot:", bot.respond("Tell me a joke"))
print("Bot:", bot.respond("How are you?"))

# Lihat history
print("\nChat History:")
for msg in history.messages:
    if isinstance(msg, HumanMessage):
        print(f"User: {msg.content}")
    else:
        print(f"Bot: {msg.content}")

Bot: Hello! How can I help you today?
Bot: I'm DummyBot with memory!
Bot: I don't understand. Can you rephrase?
Bot: I'm just a program, but I'm functioning well!

Chat History:
User: Hi
Bot: Hello! How can I help you today?
User: What's your name?
Bot: I'm DummyBot with memory!
User: Tell me a joke
Bot: I don't understand. Can you rephrase?
User: How are you?
Bot: I'm just a program, but I'm functioning well!


In [71]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("Halo!")
history.add_ai_message("Hai, ada yang bisa saya bantu?")
history.add_user_message("apa kabarnya")
history.add_ai_message("baik")
print(history.messages)

[HumanMessage(content='Halo!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hai, ada yang bisa saya bantu?', additional_kwargs={}, response_metadata={}), HumanMessage(content='apa kabarnya', additional_kwargs={}, response_metadata={}), AIMessage(content='baik', additional_kwargs={}, response_metadata={})]


In [68]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "Halo!"}, {"output": "Hai, ada yang bisa saya bantu?"})
print(memory.load_memory_variables({}))
memory

{'history': 'Human: Halo!\nAI: Hai, ada yang bisa saya bantu?'}


ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Halo!', additional_kwargs={}, response_metadata={}), AIMessage(content='Hai, ada yang bisa saya bantu?', additional_kwargs={}, response_metadata={})]))